In [ ]:
from google.colab import files
files.upload()

Saving steam_game_ids.csv to steam_game_ids.csv


{'steam_game_ids.csv': b'Game ID\r\n10\r\n20\r\n30\r\n40\r\n50\r\n60\r\n70\r\n130\r\n220\r\n240\r\n300\r\n400\r\n440\r\n500\r\n550\r\n570\r\n620\r\n730\r\n1200\r\n1250\r\n1256\r\n1257\r\n1300\r\n1313\r\n1500\r\n1510\r\n1520\r\n1530\r\n1600\r\n1610\r\n1630\r\n1640\r\n1700\r\n1930\r\n2100\r\n2200\r\n2210\r\n2270\r\n2280\r\n2310\r\n2320\r\n2500\r\n2600\r\n2610\r\n2620\r\n2630\r\n2640\r\n2710\r\n2800\r\n2810\r\n2820\r\n2840\r\n2870\r\n2910\r\n2920\r\n2990\r\n3010\r\n3020\r\n3050\r\n3130\r\n3170\r\n3270\r\n3300\r\n3310\r\n3320\r\n3330\r\n3340\r\n3350\r\n3360\r\n3380\r\n3390\r\n3410\r\n3420\r\n3430\r\n3450\r\n3460\r\n3480\r\n3500\r\n3510\r\n3520\r\n3540\r\n3560\r\n3570\r\n3590\r\n3600\r\n3610\r\n3620\r\n3700\r\n3710\r\n3720\r\n3730\r\n3800\r\n3830\r\n3900\r\n3910\r\n3920\r\n3980\r\n3990\r\n4000\r\n4300\r\n4520\r\n4570\r\n4700\r\n4800\r\n4850\r\n4870\r\n4890\r\n4920\r\n6000\r\n6010\r\n6020\r\n6030\r\n6040\r\n6060\r\n6080\r\n6090\r\n6120\r\n6200\r\n6220\r\n6300\r\n6310\r\n6400\r\n6420\r\n6510\

In [ ]:
import pandas as pd
df = pd.read_csv('steam_game_ids.csv')

In [ ]:
import requests
import time
import csv

def get_steam_game_prices(game_ids, currency='USD', language='english'):
    base_url = "https://store.steampowered.com/api/appdetails"
    results = {}

    for i, appid in enumerate(game_ids, 1):
        if i%100 == 0:
          print(f'Обработка игры {appid} ({i}/{len(game_ids)})')

        params = {
            'appids': appid,
            'cc': currency.lower(),
            'l': language.lower()
        }

        try:
            response = requests.get(base_url, params=params)
            data = response.json()

            if not data[str(appid)]['success']:
                results[appid] = {'error': 'App not found'}
                continue

            app_data = data[str(appid)]['data']

            # Основная информация об игре
            price_info = {
                'name': app_data.get('name', 'N/A'),
                'type': app_data.get('type', 'N/A'),
                'is_free': app_data.get('is_free', False),
                'packages': len(app_data.get('packages', []))
            }

            # Информация о цене для платных игр
            if not price_info['is_free'] and 'price_overview' in app_data:
                price_overview = app_data['price_overview']
                price_info['price_overview'] = {
                    'currency': price_overview.get('currency', 'N/A'),
                    'initial_price': price_overview.get('initial', 0),
                    'final_price': price_overview.get('final', 0),
                    'discount_percent': price_overview.get('discount_percent', 0),
                    'final_formatted': price_overview.get('final_formatted', 'N/A')
                }

            results[appid] = price_info

        except Exception as e:
            results[appid] = {'error': str(e)}

        time.sleep(0.5)

    return results

def start(ind):
    game_ids = df['Game ID'][1000*ind:1000*(ind+1)]
    print(f"Сбор данных для {len(game_ids)} игр...")

    prices = get_steam_game_prices(game_ids, currency='USD')

    filename = f'Steam_game_prices_dataset_{ind}.csv'

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = [
            'appid', 'name', 'type', 'is_free', 'currency',
            'initial_price', 'final_price', 'discount_percent',
            'final_formatted', 'packages_count', 'error'
        ]

        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for appid, game_data in prices.items():
            row = {'appid': appid}

            if 'error' in game_data:
                row['error'] = game_data['error']
            else:
                row.update({
                    'name': game_data.get('name', ''),
                    'type': game_data.get('type', ''),
                    'is_free': game_data.get('is_free', ''),
                    'packages_count': game_data.get('packages', '')
                })

                if game_data.get('price_overview'):
                    price = game_data['price_overview']
                    row.update({
                        'currency': price.get('currency', ''),
                        'initial_price': price.get('initial_price', ''),
                        'final_price': price.get('final_price', ''),
                        'discount_percent': price.get('discount_percent', ''),
                        'final_formatted': price.get('final_formatted', '')
                    })

            writer.writerow(row)

    print(f"Данные сохранены в файл: {filename}")

In [ ]:
for ind in range(17,27):
  start(ind)

Сбор данных для 1000 игр...
Обработка игры 2174560 (100/1000)
Обработка игры 2191490 (200/1000)
Обработка игры 2204900 (300/1000)
Обработка игры 2219890 (400/1000)
Обработка игры 2233132 (500/1000)
Обработка игры 2251010 (600/1000)
Обработка игры 2273430 (700/1000)
Обработка игры 2289630 (800/1000)
Обработка игры 2305830 (900/1000)
Обработка игры 2321103 (1000/1000)
Данные сохранены в файл: Steam_game_prices_dataset_17.csv
Сбор данных для 1000 игр...
Обработка игры 2336880 (100/1000)
Обработка игры 2351804 (200/1000)
Обработка игры 2367820 (300/1000)
Обработка игры 2383760 (400/1000)
Обработка игры 2398170 (500/1000)
Обработка игры 2415480 (600/1000)
Обработка игры 2430900 (700/1000)
Обработка игры 2447000 (800/1000)
Обработка игры 2461850 (900/1000)
Обработка игры 2481670 (1000/1000)
Данные сохранены в файл: Steam_game_prices_dataset_18.csv
Сбор данных для 1000 игр...
Обработка игры 2502940 (100/1000)
Обработка игры 2518960 (200/1000)
Обработка игры 2535830 (300/1000)
Обработка игры 2

Склеим датасеты в итоговый датасеты

In [ ]:
df0 = pd.read_csv('Steam_game_prices_dataset_0.csv')
df1 = pd.read_csv('Steam_game_prices_dataset_1.csv')
df2 = pd.read_csv('Steam_game_prices_dataset_2.csv')
df3 = pd.read_csv('Steam_game_prices_dataset_3.csv')
df4 = pd.read_csv('Steam_game_prices_dataset_4.csv')
df5 = pd.read_csv('Steam_game_prices_dataset_5.csv')
df6 = pd.read_csv('Steam_game_prices_dataset_6.csv')
df7 = pd.read_csv('Steam_game_prices_dataset_7.csv')
df8 = pd.read_csv('Steam_game_prices_dataset_8.csv')
df9 = pd.read_csv('Steam_game_prices_dataset_9.csv')
df10 = pd.read_csv('Steam_game_prices_dataset_10.csv')
df11 = pd.read_csv('Steam_game_prices_dataset_11.csv')
df12 = pd.read_csv('Steam_game_prices_dataset_12.csv')
df13 = pd.read_csv('Steam_game_prices_dataset_13.csv')
df14 = pd.read_csv('Steam_game_prices_dataset_14.csv')
df15 = pd.read_csv('Steam_game_prices_dataset_15.csv')
df16 = pd.read_csv('Steam_game_prices_dataset_16.csv')
df17 = pd.read_csv('Steam_game_prices_dataset_17.csv')
df18 = pd.read_csv('Steam_game_prices_dataset_18.csv')
df19 = pd.read_csv('Steam_game_prices_dataset_19.csv')
df20 = pd.read_csv('Steam_game_prices_dataset_20.csv')
df21 = pd.read_csv('Steam_game_prices_dataset_21.csv')
df22 = pd.read_csv('Steam_game_prices_dataset_22.csv')
df23 = pd.read_csv('Steam_game_prices_dataset_23.csv')
df24 = pd.read_csv('Steam_game_prices_dataset_24.csv')
df25 = pd.read_csv('Steam_game_prices_dataset_25.csv')
df26 = pd.read_csv('Steam_game_prices_dataset_26.csv')
df_total = [df0, df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16, df17, df18, df19, df20, df21, df22, df23, df24, df25, df26]

In [ ]:
df_price = pd.concat(df_total, ignore_index= True)

In [ ]:
df_price.to_csv('steam_game_prices_dataset.csv')